In [1]:
!pip install earthengine-api

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

In [3]:
import ee
ee.Initialize()

# Define a region of interest

# Get the first image
image = bands.first()

# Print image info
print(image.getInfo())


{'type': 'Image', 'bands': [{'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [6991, 6911], 'crs': 'PROJCS["PS         WGS84", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Polar_Stereographic"], \n  PARAMETER["central_meridian", 0.0], \n  PARAMETER["latitude_of_origin", -71.0], \n  PARAMETER["scale_factor", 1.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [30, 0, -1173615, 0, -30, 1901715]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [6991, 6911], 'crs': 'PROJCS["PS        

In [4]:
!pip install folium

In [36]:
import folium
import numpy as np

roi_center = [-70, -33.8719]
roi = ee.Geometry.Point(roi_center)

# Load Landsat 8 image collection, filter by date and region
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate('2024-01-24', '2024-02-04').filterBounds(roi)

bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11']
image_all_bands = landsat.median().select(bands)

vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B1'],
}


# Create a folium map centered at the region of interest
# (es necesario el np.flip, las coordenadas estan invertidas en relacion al otro)
m = folium.Map(location=np.flip(roi_center), zoom_start=7)

# Add the image layer to the map
map_id_dict = ee.Image(image_all_bands).getMapId(vis_params)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='Landsat 8 True Color',
    overlay=True,
    control=True
).add_to(m)

print(map_id_dict['tile_fetcher'].url_format)

# Add layer control
m.add_child(folium.LayerControl())

# Display the map
m

https://earthengine.googleapis.com/v1/projects/632253003966/maps/6e9791510053c2bf9a48fc532da7d7a9-c6eab2b28c786a459b2d7e3098c060bf/tiles/{z}/{x}/{y}
